In [1]:
import time
import json
import numpy as np
import pandas as pd
import covasim as cv
from pyDOE import lhs
from src.data import get_regional_data
from src.interventions import get_sampling_interventions

Covasim 2.0.3 (2021-03-11) — © 2021 by IDM


# Data Generation

## Initial Configuration

Once the dataset is calibrated, we would like to use it to predict possible future evolutions of the pandemic.

Given that *Covasim* needs a warm-up period to reach a stabler pandemic condition, we will proceed by generating the dataset in this way:
* we run a simulation for a total of *zones_starting_day $+$ num_zones $\cdot$ time_interval days*
* we discard the first *zones_starting_day* days (251, in our case), meaning that we will collect data from 01/11/2020 on, which is when the evolution of the pandemic started having a behaviour more similar to the one we have right now (i.e., after the summer period)
* we aggregate the data on a time window of size *2 $\cdot$ time_interval* (6 weeks, in our case), in which we consider the first half of the data as "inputs", and the last half as "outputs"
  * right in the middle of these windows there is a zone change
  * it will be used as input of the surrogate model together with the other data

In this way, we aim at maximizing the number of samples that we can get from each run, while trying at the same time to cover the data space in a (hopefully) almost uniform way.

In [2]:
num_zones = 8
time_interval = 21
zones_starting_day = 251

We can now retrieve the parameters' configuration that we obtained from the calibration step and set the length of the simulation from the previously set values.

In [3]:
with open('../res/parameters.json', 'r') as json_file:
    j = json.load(json_file)

intervention_params = j['intervention_params']
initial_params = j['initial_params']
initial_params['n_days'] = zones_starting_day + num_zones * time_interval
df = get_regional_data(4.46e6 / initial_params['pop_size'])

initial_params

{'pop_type': 'hybrid',
 'location': 'italy',
 'start_day': '2020-02-24',
 'pop_size': 400000,
 'pop_infected': 720,
 'n_beds_hosp': 1481.6,
 'n_beds_icu': 57.84,
 'quar_period': 14,
 'verbose': 0,
 'n_days': 419}

## Latin-Hypercube Sampling

We have *4* possible categories for the zones, and each run of the simulator will consider a sequence of *8* zone changes. Thus, the total number of possible sequences is $4^8 = 65536$, which, if we consider an average of 2 minutes for each run, will end the generation phase after 90 days.

This is clearly not desirable, thus we rely on *Latin-Hypercube Sampling* to generate a subset of *300* sequences which, however, is supposed to cover the sequences space almost uniformly. Again, if we consider an average of 2 minutes for each run, will end the generation phase after 10 hours.

In [4]:
np.random.seed(42)
samples = lhs(n=num_zones, samples=200)
samples = pd.DataFrame(samples)
for i in range(num_zones):
    samples[i] = samples[i].map(lambda v: int(4 * v)).map({0: 'W', 1: 'Y', 2: 'O', 3: 'R'})
samples

,0,1,2,3,4,5,6,7
0,Y,R,W,Y,R,Y,Y,Y
1,Y,O,O,Y,R,Y,R,Y
2,Y,O,W,O,R,O,W,O
3,O,W,W,W,Y,R,R,Y
4,R,Y,O,O,R,O,R,W
...,...,...,...,...,...,...,...,...
195,O,Y,O,R,Y,O,O,W
196,Y,R,Y,R,W,R,O,Y
197,O,O,O,R,O,W,W,W
198,Y,Y,O,O,O,W,W,R


We can start the generation script, from which we will collect the data about *hospitalized* individuals and new *diagnosed* cases and *deaths*.

In [5]:
data = []
for idx, zones in samples.iterrows():
    print(f'Generating samples for simulation {idx + 1:0{len(str(samples.shape[0]))}}/{samples.shape[0]}', end='')
    start_time = time.time()
    intervs = get_sampling_interventions(zones, intervention_params, time_interval)
    sim = cv.Sim(pars={**initial_params, 'rand_seed': idx}, interventions=intervs, datafile=df)
    sim.run()
    # concatenate data in an array of shape (num_days, 4)
    result = np.concatenate((
        sim.results['n_severe'].values + sim.results['n_critical'].values,
        sim.results['new_diagnoses'].values,
        sim.results['new_deaths'].values
    )).reshape(3, -1).transpose()
    # retrieve data about last num_zones * time_interval and flatten it to get num_zones rows
    result = result[-num_zones * time_interval:].reshape(num_zones, -1)
    # get data about two subsequent zones with respective zone colors
    data += [np.concatenate((inp, out, zones[z:z + 2])) for z, (inp, out), in enumerate(zip(result[:-1], result[1:]))]
    print(f' -- elapsed time: {time.time() - start_time:.4}s')

Generating samples for simulation 001/200 -- elapsed time: 120.9s
Generating samples for simulation 002/200 -- elapsed time: 98.75s
Generating samples for simulation 003/200 -- elapsed time: 90.75s
Generating samples for simulation 004/200 -- elapsed time: 89.6s
Generating samples for simulation 005/200 -- elapsed time: 92.26s
Generating samples for simulation 006/200 -- elapsed time: 89.2s
Generating samples for simulation 007/200 -- elapsed time: 90.86s
Generating samples for simulation 008/200 -- elapsed time: 88.11s
Generating samples for simulation 009/200 -- elapsed time: 87.59s
Generating samples for simulation 010/200 -- elapsed time: 81.26s
Generating samples for simulation 011/200 -- elapsed time: 88.63s
Generating samples for simulation 012/200 -- elapsed time: 87.81s
Generating samples for simulation 013/200 -- elapsed time: 86.8s
Generating samples for simulation 014/200 -- elapsed time: 86.69s
Generating samples for simulation 015/200 -- elapsed time: 90.19s
Generating sa

## Saving the Dataset

We can now collect the data into a pandas dataframe and store it in a *csv* file to be used to train the surrogate model.

In [6]:
pd.options.display.max_columns = 2 * time_interval + 2

columns = [f'{c}_{d}' for d in range(0, 2 * time_interval) for c in ['hosp', 'diag', 'dead']]
data = pd.DataFrame(data, columns=columns + ['init_zone', 'actuated_zone'])
data.to_csv('../res/dataset.csv', index=False)
data

,hosp_0,diag_0,dead_0,hosp_1,diag_1,dead_1,hosp_2,diag_2,dead_2,hosp_3,diag_3,dead_3,hosp_4,diag_4,dead_4,hosp_5,diag_5,dead_5,hosp_6,diag_6,dead_6,hosp_7,...,diag_35,dead_35,hosp_36,diag_36,dead_36,hosp_37,diag_37,dead_37,hosp_38,diag_38,dead_38,hosp_39,diag_39,dead_39,hosp_40,diag_40,dead_40,hosp_41,diag_41,dead_41,init_zone,actuated_zone
0,98.0,49.0,1.0,106.0,118.0,1.0,107.0,89.0,3.0,116.0,102.0,0.0,125.0,114.0,0.0,134.0,89.0,2.0,137.0,76.0,2.0,151.0,...,33.0,7.0,180.0,43.0,3.0,173.0,19.0,5.0,167.0,47.0,2.0,151.0,44.0,5.0,143.0,36.0,2.0,135.0,19.0,1.0,Y,R
1,224.0,95.0,1.0,219.0,135.0,7.0,223.0,128.0,4.0,227.0,111.0,3.0,229.0,122.0,5.0,222.0,113.0,6.0,224.0,68.0,6.0,230.0,...,30.0,0.0,61.0,85.0,1.0,68.0,59.0,0.0,69.0,63.0,3.0,76.0,59.0,0.0,78.0,28.0,1.0,83.0,81.0,1.0,R,W
2,119.0,22.0,3.0,109.0,17.0,2.0,100.0,23.0,1.0,91.0,26.0,1.0,85.0,27.0,0.0,75.0,26.0,5.0,68.0,10.0,2.0,69.0,...,61.0,2.0,192.0,168.0,5.0,186.0,171.0,5.0,194.0,138.0,5.0,201.0,129.0,3.0,199.0,146.0,2.0,196.0,112.0,6.0,W,Y
3,94.0,50.0,0.0,96.0,99.0,2.0,106.0,100.0,0.0,111.0,73.0,2.0,117.0,105.0,1.0,124.0,85.0,2.0,132.0,80.0,3.0,146.0,...,21.0,5.0,203.0,62.0,2.0,188.0,36.0,5.0,157.0,33.0,9.0,152.0,27.0,3.0,144.0,21.0,1.0,138.0,15.0,5.0,Y,R
4,193.0,65.0,6.0,198.0,170.0,2.0,199.0,145.0,5.0,206.0,169.0,3.0,211.0,113.0,5.0,211.0,109.0,1.0,209.0,69.0,6.0,208.0,...,15.0,1.0,31.0,48.0,0.0,31.0,41.0,1.0,34.0,38.0,0.0,34.0,36.0,1.0,35.0,28.0,2.0,36.0,33.0,1.0,R,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,128.0,17.0,5.0,121.0,23.0,2.0,104.0,14.0,4.0,89.0,6.0,4.0,75.0,20.0,5.0,76.0,12.0,2.0,69.0,20.0,1.0,64.0,...,3.0,0.0,5.0,12.0,0.0,5.0,10.0,0.0,8.0,9.0,0.0,9.0,7.0,0.0,9.0,7.0,0.0,10.0,14.0,0.0,R,O
1396,15.0,2.0,0.0,14.0,1.0,0.0,13.0,1.0,0.0,11.0,0.0,0.0,10.0,3.0,0.0,9.0,2.0,0.0,8.0,1.0,0.0,7.0,...,3.0,1.0,19.0,10.0,1.0,20.0,4.0,0.0,20.0,3.0,0.0,20.0,3.0,0.0,18.0,1.0,0.0,13.0,0.0,1.0,O,R
1397,10.0,7.0,0.0,13.0,16.0,0.0,16.0,12.0,0.0,16.0,13.0,0.0,17.0,12.0,0.0,18.0,8.0,0.0,23.0,7.0,0.0,23.0,...,12.0,0.0,10.0,19.0,1.0,12.0,16.0,0.0,13.0,29.0,0.0,14.0,17.0,0.0,15.0,11.0,0.0,14.0,13.0,1.0,R,W
1398,11.0,1.0,1.0,6.0,2.0,1.0,5.0,2.0,0.0,4.0,2.0,0.0,4.0,0.0,0.0,4.0,5.0,0.0,4.0,4.0,0.0,4.0,...,18.0,1.0,41.0,29.0,0.0,46.0,21.0,1.0,49.0,26.0,0.0,47.0,14.0,1.0,46.0,21.0,2.0,36.0,9.0,5.0,W,O
